In [1]:
import sys

sys.path.append('../')

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
import HuichuanFlow as ms

# 加载MNIST数据集，只取5000个样本
from sklearn.datasets import load_digits

mnist = load_digits()

# 加载MNIST数据集，只取5000个样本
X, y = mnist.data, mnist.target

X, y = X[:5000] / 255, y.astype(np.int)[:5000]

In [2]:
# 将整数形式的标签转换成One-Hot编码
oh = OneHotEncoder(sparse=False)
one_hot_label = oh.fit_transform(y.reshape(-1, 1))

In [3]:
# 输入图像尺寸
img_shape = (8, 8)

In [16]:
# 输入图像
x = ms.core.Variable(img_shape, init=False, trainable=False)

# One-Hot标签
one_hot = ms.core.Variable(dim=(10, 1), init=False, trainable=False)

# 第一卷积层
conv1 = ms.layer.conv([x], img_shape, 3, (2, 2), "ReLU")

# 第一池化层
pooling1 = ms.layer.pooling(conv1, (2, 2), (2, 2))

# 第二卷积层
conv2 = ms.layer.conv(pooling1, (4, 4), 10, (3, 3), "ReLU")

# 第二池化层
pooling2 = ms.layer.pooling(conv2, (2, 2), (2, 2))


# 全连接层
fc1 = ms.layer.fc(ms.ops.Concat(*pooling2), 40, 10, "ReLU")

# # 输出层
# output = ms.layer.fc(fc1, 20, 10, "None")

# 分类概率
predict = ms.ops.SoftMax(fc1)

# 交叉熵损失
loss = ms.ops.loss.CrossEntropyWithSoftMax(output, one_hot)

# 学习率
learning_rate = 0.1

# 优化器
optimizer = ms.optimizer.Adam(ms.default_graph, loss, learning_rate)

# 批大小
batch_size = 128


In [17]:
# 训练
for epoch in range(60):
    
    batch_count = 0
    
    for i in range(len(X)):
        
        feature = np.mat(X[i]).reshape(img_shape)
        label = np.mat(one_hot_label[i]).T
        
        x.set_value(feature)
        one_hot.set_value(label)
        

        optimizer.one_step()
        

        batch_count += 1
        if batch_count >= batch_size:
            
            print("epoch: {:d}, iteration: {:d}, loss: {:.3f}".format(epoch + 1, i + 1, loss.value[0, 0]))

            optimizer.update()
            batch_count = 0
        

    pred = []
    for i in range(len(X)):
                
        feature = np.mat(X[i]).reshape(img_shape)
        x.set_value(feature)
        
        predict.forward()
        pred.append(predict.value.A.ravel())
            
    pred = np.array(pred).argmax(axis=1)
    accuracy = (y == pred).astype(np.int).sum() / len(X)
       
    print("epoch: {:d}, accuracy: {:.3f}".format(epoch + 1, accuracy))


epoch: 1, iteration: 128, loss: 2.252
epoch: 1, iteration: 256, loss: 3.779
epoch: 1, iteration: 384, loss: 3.500
epoch: 1, iteration: 512, loss: 0.685
epoch: 1, iteration: 640, loss: 2.441
epoch: 1, iteration: 768, loss: 2.609
epoch: 1, iteration: 896, loss: 3.341
epoch: 1, iteration: 1024, loss: 1.845
epoch: 1, iteration: 1152, loss: 1.872
epoch: 1, iteration: 1280, loss: 2.426
epoch: 1, iteration: 1408, loss: 2.583
epoch: 1, iteration: 1536, loss: 2.458
epoch: 1, iteration: 1664, loss: 2.816
epoch: 1, iteration: 1792, loss: 2.177
epoch: 1, accuracy: 0.098
epoch: 2, iteration: 128, loss: 2.265
epoch: 2, iteration: 256, loss: 2.283
epoch: 2, iteration: 384, loss: 3.004
epoch: 2, iteration: 512, loss: 2.316
epoch: 2, iteration: 640, loss: 2.322
epoch: 2, iteration: 768, loss: 2.597
epoch: 2, iteration: 896, loss: 2.294
epoch: 2, iteration: 1024, loss: 2.428
epoch: 2, iteration: 1152, loss: 2.197
epoch: 2, iteration: 1280, loss: 2.329
epoch: 2, iteration: 1408, loss: 2.510
epoch: 2, ite

KeyboardInterrupt: 